In [1]:
import os
import torch
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from tqdm.notebook import tqdm

# 📁 Cartella pazienti
PT_DIR = "/content/drive/MyDrive/NeuroOnco/Derivate"

# 📌 Selezione file pazienti
pt_files = sorted([f for f in os.listdir(PT_DIR) if f.endswith(".pt")])
select_files = widgets.SelectMultiple(
    options=pt_files,
    description="Pazienti:",
    layout=widgets.Layout(width="60%", height="200px")
)

run_button = widgets.Button(description="Estrai dati", button_style="success")
output = widgets.Output()
display(widgets.VBox([select_files, run_button, output]))

# 📊 Estrazione statistica per un volume ROI
def calcola_statistiche(valori):
    valori = valori.astype(np.float64)
    valori = valori[np.isfinite(valori)]
    valid_mask = (valori > -1e5) & (valori < 1e5)
    valori = valori[valid_mask]

    if len(valori) == 0:
        return {}

    return {
        "count": len(valori),
        "mean": np.mean(valori),
        "std": np.std(valori),
        "min": np.min(valori),
        "25%": np.percentile(valori, 25),
        "50%": np.median(valori),
        "75%": np.percentile(valori, 75),
        "max": np.max(valori),
        "skew": pd.Series(valori).skew(),
        "kurtosis": pd.Series(valori).kurtosis()
    }

# 📥 Callback
def on_run(b):
    with output:
        clear_output()
        risultati = []
        print(f"🔍 Analizzo {len(select_files.value)} pazienti...")

        for fname in tqdm(select_files.value):
            path = os.path.join(PT_DIR, fname)
            try:
                data = torch.load(path, map_location="cpu")
                vol = data["volume"].numpy()
                roi_masks = data.get("roi_masks", {})
                nomi_serie = data.get("nomi_serie", [f"Serie_{i}" for i in range(vol.shape[0])])

                for serie_idx, serie_name in enumerate(nomi_serie):
                    serie_data = vol[serie_idx]

                    for roi_name in roi_masks:
                        mask = roi_masks[roi_name].numpy().astype(bool)
                        valori = serie_data[mask]
                        stats = calcola_statistiche(valori)

                        if stats:
                            stats_row = {
                                "paziente": fname.replace(".pt", ""),
                                "roi": roi_name,
                                "serie": serie_name
                            }
                            stats_row.update(stats)
                            risultati.append(stats_row)

            except Exception as e:
                print(f"❌ Errore con {fname}: {e}")

        df = pd.DataFrame(risultati)
        display(df)
        print("✅ Completato.")

        # 📁 Salva CSV
        df.to_csv("/content/statistiche_roi_coorte.csv", index=False)

run_button.on_click(on_run)


GRAFICI

In [2]:
import pandas as pd

df = pd.read_csv("/content/statistiche_roi_coorte.csv")


FileNotFoundError: [Errno 2] No such file or directory: '/content/statistiche_roi_coorte.csv'

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import re

def plot_median_with_quartiles(df):
    pazienti = df['paziente'].unique()
    serie_order = sorted(df['serie'].unique())

    for paziente in pazienti:
        df_p = df[df['paziente'] == paziente]

        for serie in serie_order:
            df_s = df_p[df_p['serie'] == serie]
            if df_s.empty:
                continue

            # Filtra solo le ROI con MALATO-OPx o DUBBIA-OPx
            roi_valide = [roi for roi in df_s['roi'].unique() if re.match(r'^(MALATO|DUBBIA)-OP\d+$', roi)]

            # Ordina: prima i MALATO-OPx, poi i DUBBIA-OPx
            def ordine_roi(roi):
                tipo, op = roi.split('-')
                return (0 if tipo == "MALATO" else 1, int(op.replace("OP", "")))

            roi_order = sorted(roi_valide, key=ordine_roi)

            df_s = df_s[df_s['roi'].isin(roi_order)].set_index('roi').reindex(roi_order).reset_index()

            if df_s.empty or not {'25%', '50%', '75%'}.issubset(df_s.columns):
                print(f"⚠️ Dati incompleti per {paziente} – {serie}. Salto.")
                continue

            x = np.arange(len(roi_order))
            y = df_s['50%'].values.astype(np.float32)
            yerr_lower = y - df_s['25%'].values.astype(np.float32)
            yerr_upper = df_s['75%'].values.astype(np.float32) - y
            yerr = np.array([yerr_lower, yerr_upper])

            plt.figure(figsize=(10, 5))
            plt.bar(x, y, yerr=yerr, capsize=5, align='center', alpha=0.7)
            plt.xticks(x, roi_order, rotation=45)
            plt.title(f"Mediana e Quartili – Paziente: {paziente} | Serie: {serie}")
            plt.ylabel("Valori voxel")
            plt.grid(True)
            plt.tight_layout()
            plt.show()



In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from tqdm.notebook import tqdm

# 📁 Cartella con i .pt
PT_DIR = "/content/drive/MyDrive/NeuroOnco/Derivate"

# 🔘 Widget selezione pazienti
pt_files = sorted([f for f in os.listdir(PT_DIR) if f.endswith(".pt")])
select_files = widgets.SelectMultiple(
    options=pt_files,
    value=[],
    description="Pazienti:",
    layout=widgets.Layout(width="60%", height="200px")
)
run_button = widgets.Button(description="Genera Boxplot", button_style="success")
output = widgets.Output()
display(widgets.VBox([select_files, run_button, output]))

# 📊 Funzione principale
def on_click(b):
    with output:
        clear_output()
        print("📊 Estrazione in corso...")

        risultati = []

        for fname in tqdm(select_files.value):
            path = os.path.join(PT_DIR, fname)
            try:
                data = torch.load(path, map_location="cpu")
                vol = data["volume"].numpy()
                roi_masks = data.get("roi_masks", {})
                nomi_serie = data.get("nomi_serie", [f"Serie_{i}" for i in range(vol.shape[0])])
                paziente = fname.replace(".pt", "")

                for serie_idx, serie_name in enumerate(nomi_serie):
                    serie_data = vol[serie_idx]

                    for roi_key in ["MALATO_COM", "REF"]:
                        if roi_key in roi_masks:
                            mask = roi_masks[roi_key].numpy().astype(bool)
                            valori = serie_data[mask]
                            valori = valori[np.isfinite(valori)]

                            # 🔍 Escludi -1000 sempre, -10 solo per APT
                            valori = valori[valori != -1000]
                            if "APT" in serie_name.upper():
                                valori = valori[valori != -10]

                            # 🔁 Applica trasformazione per KTRAN (log)
                            if "KTRAN" in serie_name.upper():
                                valori = valori[valori > 0]  # log valido solo > 0
                                valori = np.log(valori)

                            # 🔁 Normalizza T1W sul valore medio della ROI EXTERNAL
                            elif "T1W" in serie_name.upper() and "EXTERNAL" in roi_masks:
                                mask_ext = roi_masks["EXTERNAL"].numpy().astype(bool)
                                base_vals = serie_data[mask_ext]
                                base_vals = base_vals[np.isfinite(base_vals) & (base_vals != -1000)]
                                if base_vals.size > 0:
                                    mean_ext = base_vals.mean()
                                    valori = valori / mean_ext

                            for v in valori:
                                risultati.append({
                                    "paziente": paziente,
                                    "serie": serie_name,
                                    "roi": roi_key,
                                    "valore": float(v)
                                })


            except Exception as e:
                print(f"❌ Errore con {fname}: {e}")

        # 📋 Costruzione DataFrame
        df = pd.DataFrame(risultati)
        if df.empty:
            print("⚠️ Nessun dato valido trovato.")
            return

        # 📈 Boxplot per ogni sequenza
        serie_order = sorted(df['serie'].unique())
        n_col = 3
        n_row = int(np.ceil(len(serie_order) / n_col))
        fig, axes = plt.subplots(n_row, n_col, figsize=(5 * n_col, 5 * n_row), squeeze=False)

        for i, serie in enumerate(serie_order):
            ax = axes[i // n_col][i % n_col]
            df_s = df[df['serie'] == serie]

            sns.boxplot(
                data=df_s,
                x="paziente",
                y="valore",
                hue="roi",
                ax=ax
            )

            ax.set_title(f"Serie: {serie}")
            ax.set_xlabel("Paziente")
            ax.set_ylabel("Valore voxel")
            ax.tick_params(axis='x', rotation=45)
            ax.grid(True)

        # Rimuove subplot vuoti
        for j in range(len(serie_order), n_row * n_col):
            fig.delaxes(axes[j // n_col][j % n_col])

        fig.tight_layout()
        plt.show()


        # 📊 Calcolo statistiche riassuntive e salvataggio per abstract
        df_stats = (
            df.groupby(['paziente', 'serie', 'roi'])['valore']
            .agg(['count', 'mean', 'std', 'min', 'median', 'max',
                  lambda x: np.percentile(x, 25),
                  lambda x: np.percentile(x, 75),
                  pd.Series.skew, pd.Series.kurtosis])
            .reset_index()
        )
        df_stats.columns = ['paziente', 'serie', 'roi', 'count', 'mean', 'std', 'min', 'median', 'max',
                            'q25', 'q75', 'skew', 'kurtosis']

        df_stats.to_csv("/content/interpretazione_roi_neuroonco.csv", index=False)
        print("✅ File salvato: /content/interpretazione_roi_neuroonco.csv")


run_button.on_click(on_click)
